In [3]:
import numpy as np

In [151]:
def compute_1st_part_y_t(z_matrix, rho_matrix, n_observations):
    n_observations = z_matrix.shape[1]
    j_obs = np.arange(0,n_observations)
    return np.dot(z_matrix* rho_matrix, j_obs)

def compute_2nd_part_y_t(z_matrix, rho_matrix, x_vector):
    return np.sum(z_matrix *(1-rho_matrix)* x_vector, axis = 1)

def compute_y_t(z_matrix, rho_matrix, x_vector):
    t, n_obs = z_matrix.shape
    y_t_1 = compute_1st_part_y_t(z_matrix, rho_matrix, n_obs)
    y_t_2 = compute_2nd_part_y_t(z_matrix, rho_matrix, x_vector)
    y_t = (y_t_1 + y_t_2).reshape(t,1)
    return y_t
def state_attraction_repulsion_f1(alpha_t,beta_t,c):
    #state: int state in (0,n_states-1)
    #alpha_t: np.array shape(n_states,)
    #beta_t: np.array shape(n_states,)
    #c int
    numerator = alpha_t + beta_t
    denominator = np.sum(numerator ,axis=0)
    result = c * (numerator - denominator)
    return result

def f2_function(z_matrix, x_obs_vector):
    t = z_matrix.shape[0]
    z_vector = np.where(z_matrix == 1)[1].reshape(t,1)
    vec_diff = z_vector - x_obs_vector
    return np.count_nonzero(vec_diff)

def utility_u1_state_attraction_repulsion_function(alpha, beta, c, w1, w2, z_matrix, x_obs_vector):
    u_part_1 = w1 * state_attraction_repulsion_f1(alpha, beta, c)
    u_part_2 = w2 * f2_function(z_matrix, x_obs_vector)
    return u_part_1 + u_part_2

In [118]:
Z_M = np.array([[1, 0, 0],
                [0, 1, 0],
                [1, 0, 0],
                [0, 0, 1],
                [1, 0, 0],
                [0, 1, 0]])

rho_M = np.array([[1,1,1],
                 [1,0,1],
                 [0,1,1],
                 [0,1,0],
                 [0,0,1],
                 [1,0 ,0]])


X_M = np.array([[0],
               [1],
               [0],
               [2],
               [0],
               [1]])

create_1st_part_y_t(Z_M,rho_M)+create_2nd_part_y_t(Z_M, rho_M, X_M)

array([1, 1, 0, 2, 0, 1])

#### HMM

In [119]:
from hmm_utils import *

In [120]:
class HMM(hmm.MultinomialHMM):

    '''
    Builts HMM
    Parameters
    ----------
    n_components : int
        Number of hidden states
    '''
    
    def __init__(self, n_components):
        
        super().__init__(n_components)
  
    def smoothing(self, X, t):
        return self.predict_proba(X)[t]
    
    def alpha(self, X):

        alphas = self._do_forward_pass( 
            self._compute_log_likelihood(X) )[1]

        return alphas
    
    def beta(self, X):
        
        betas = self._do_backward_pass( 
            self._compute_log_likelihood(X) ) #REMOVE 0 FROM BETA
            
        return betas
    
    def nu(self, X):
        V   = np.zeros([len(X), self.n_components]) 
        Ptr = np.zeros([len(X), self.n_components]) 

        lemissionprob_ = np.log(self.emissionprob_)
        ltransmat_     = np.log(self.transmat_)
        lstartprob_   = np.log(self.startprob_)

        # Init
        V[0] = lemissionprob_[:, X[0].item()] + lstartprob_

        # Forward
        for t in np.arange(1 , len(X) ):

            V[t]   = ( lemissionprob_[:, X[t].item()] + 
                np.max( ltransmat_ + V[t-1], axis=1 ) )

            Ptr[t] =  np.argmax( ltransmat_ + V[t-1], axis=1 )

        # Backward

        z_max = np.ones(len(X), dtype=int)

        z_max[-1] = np.argmax(V[-1])

        for t in np.arange(len(X)-2, -1, -1):    
            z_max[t] = Ptr[t+1, z_max[t+1]]
            
        return V, z_max


    def sample_mat(self, p, n=1, k=1000):
        '''
        Sample transition/emission matrix
        Parameters
        ----------
        p : numpy.array
            Each row is the mean of the Dirichlet we sample from.
        n : int
            Number of samples
        k : int
            The larget k the smaller the variance
        '''

        mat = np.zeros( [n, p.shape[0], p.shape[1]] )

        for i in range(n):

            mat[i] = np.apply_along_axis(lambda x: 
                np.random.dirichlet(x, 1), 1, k*p).reshape(p.shape[0],-1)

        return mat
    
    def state_attraction_repulsion_f1(self,X,c):  ##NEED TO CORRECT
        #compute in log probabilities terms
        alpha_t_i = self.alpha(X) 
        beta_t_i = self.beta(X)
        numerator = alpha_t_i + beta_t_i
        denominator = 1/( np.sum(alpha_t_i + beta_t_i ,axis=1))
        return c*((numerator.T * denominator).T)

In [121]:
priors     = np.array([0.5,0.3,0.2])
transition = np.array([[0.85, 0.05,0.1],
                       [0.05, 0.9,0.05],
                     [1/2, 1/4, 1/4]])
emission   = np.array([[1/6, 1/6, 1/6, 1/6, 1/6, 1/6],
                    [0.1, 0.1, 0.1, 0.1, 0.1, 0.5],
                       [0.1, 0.1, 0.1, 0.1, 0.1, 0.5]])

m = HMM(3)

m.startprob_ = priors
m.transmat_ = transition
m.emissionprob_ = emission

In [130]:
Z_M  = np.array([[1, 0, 0],
                [0, 1, 0],
                [1, 0, 0],
                [0, 0, 1],
                [1, 0, 0]])

X_V = m.sample(5)[0]


rho_M = np.array([[1,1,1],
                      [0,1,1],
                      [1,0,1],
                      [1,0,1],
                      [0,0,1]])

compute_y_t(Z_M, rho_M, X_V)

array([[0],
       [1],
       [0],
       [2],
       [0]], dtype=int64)

In [148]:
compute_y_t(Z_M, rho_M, X_V) - np.where(Z_M ==1)[1].reshape(5,1)

array([[0],
       [0],
       [0],
       [0],
       [0]], dtype=int64)

In [149]:
f2_function(Z_M, X_V)

3